# SimResults To Race Reports and Championship Tables

In [1]:
import pandas as pd
import numpy as np
import re, os

### Race Report Generator

In [2]:
class RaceReport():
    
    # Race report table format strings
    qualy_row_0 = """(% border="1" style="width:554px" %)"""
    qualy_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 233px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 77px; text-align: center;" %)Time"""
    qualy_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:77px" %){}"""
    qualy_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:77px" %){}"""
    qualy_row_last = """|=(% colspan="5" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 552px;" %)[[Source>>{}]]"""
    race_row_0 = """(% border="1" style="width:747px" %)"""
    race_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 229px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)Laps|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 111px; text-align: center;" %)Time/Retired|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 53px; text-align: center;" %)Grid|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 62px; text-align: center;" %)Points"""
    race_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:44px" %){}|(% style="text-align:center; border-color:#000000; width:111px" %){}|(% style="text-align:center; border-color:#000000; width:53px" %){}|(% style="text-align:center; border-color:#000000; width:62px" %){}"""
    race_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:44px" %){}|(% style="text-align:center; width:111px" %){}|(% style="text-align:center; width:53px" %){}|(% style="text-align:center; width:62px" %){}"""
    race_row_last = """|=(% colspan="8" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 745px;" %)[[Source>>{}]]"""
    race_row_fastestlap = """Fastest lap:  [[image:{}||height="14" width="23"]] {} - {}"""
    
    
    # Read series drivers info
    def read_drivers_table(series_directory):
        drivers_table_file = "{}/drivers_table.csv".format(series_directory)
        table = pd.read_csv(drivers_table_file, dtype=object)
        table = table.set_index("ign")
        return table
    
    
    # Read series points scoring info
    def read_points_table(series_directory):
        points_table_file = "{}/points_table.csv".format(series_directory)
        table = pd.read_csv(points_table_file, dtype=object)
        table = table.astype(int)
        table = table.set_index("pos")
        return table
    
    
    # Read series track info (used only for championship table)
    def read_tracks_table(series_directory):
        tracks_table_file = "{}/tracks_table.csv".format(series_directory)
        table = pd.read_csv(tracks_table_file, dtype=object)
        table["csv_manual_adjustment"] = table["csv_manual_adjustment"].astype(int)
        table = table.set_index("directory")
        return table
    
    
    # Constructor, optionally pass in already-parsed drivers and points table info, debug to print intermediate dataframes
    def __init__(self, table_names, series_directory, race_directory, drivers_table=None, points_table=None, csv_manual_adjustment=0, output_file_name="wiki_tables.txt", debug=False):
        
        self.table_names = table_names
        self.debug = debug
        
        if self.debug: print("Creating race report:", series_directory, race_directory)
        
        self.drivers_table = drivers_table if drivers_table is not None else RaceReport.read_drivers_table(series_directory)
        self.points_table = points_table if points_table is not None else RaceReport.read_points_table(series_directory)

        race_directory_path = "{}/{}".format(series_directory, race_directory)
        assert os.path.isdir(race_directory_path), "Race directory path does not exist: {}".format(race_directory_path)
        
        self.output_file = "{}/{}".format(race_directory_path, output_file_name)
        
        simresults_file_name = [file for file in os.listdir(race_directory_path) if file.endswith(".csv")][0]
        simresults_code = os.path.splitext(simresults_file_name)[0]
        self.results_file = "{}/{}.csv".format(race_directory_path, simresults_code)
        self.simresults_url = "https://simresults.net/{}".format(simresults_code)
        
        self.csv_manual_adjustment = csv_manual_adjustment
        self.tables = self.__read_results_tables()
        self.__clean_results_tables()
    
    
    # Return race result pandas dataframes in dict keyed by table_names
    def __read_results_tables(self):
        
        rows = {name: [0,0] for name in self.table_names}
        with open(self.results_file) as fp:
            current_table = ""
            for i, row in enumerate(fp):
                if current_table:
                    if row=="\n":
                        rows[current_table][1] = i
                        if self.debug: print("ending line {}".format(rows[current_table][1]))
                        current_table = ""
                else:
                    for name in self.table_names:
                        if row.startswith(name):
                            current_table = name
                            rows[current_table][0] = i+2
                            if self.debug: print("found table '{}' starting line {} ".format(current_table, rows[current_table][0]), end="")
        
        tables = {}
        for name in self.table_names:
            table_skiprows = rows[name][0] + self.csv_manual_adjustment
            table_nrows = rows[name][1] - table_skiprows - 1 + self.csv_manual_adjustment
            table_df = pd.read_csv(self.results_file, skiprows=table_skiprows, nrows=table_nrows, index_col=False, dtype=object)
            if self.debug: display(table_df)
            tables[name] = table_df
        
        return tables
    
    
    # Clean race result dataframes, cast numerical columns to integers, join points information and starting positions to driver rows
    def __clean_results_tables(self):
        
        # Merge starting position info to grid column of a race table from either quali session or previous race, optionally adding quali points column
        def merge_qualy_info(race_table, qualy_table, add_qualy_points=False):
            
            qualy_table = qualy_table.reset_index()
            race_table = race_table.reset_index()
            
            if add_qualy_points:
                qualy_table = qualy_table[["Pos", "Driver", "Points"]]
                qualy_table = qualy_table.rename(columns={"Pos":"Grid", "Points":"Qualify Points"})
            else:
                qualy_table = qualy_table[["Pos", "Driver"]]
                qualy_table = qualy_table.rename(columns={"Pos":"Grid"})

            race_table = race_table.merge(qualy_table, how='left', on="Driver")

            race_table["Grid"] = race_table["Grid"].fillna(-1)
            # Pandas issue where NaN values cause ints to become floats?
            race_table["Grid"] = race_table["Grid"].astype(int)
            race_table["Pos"] = race_table["Pos"].astype(int)
            race_table["Laps"] = race_table["Laps"].astype(int)

            if add_qualy_points:
                race_table["Qualify Points"] = race_table["Qualify Points"].fillna(0)
                race_table["Qualify Points"] = race_table["Qualify Points"].astype(int)
                
            race_table = race_table.set_index("Pos")

            return race_table
        
        # Strip quotes and whitespace from strings, cast position and laps to integers, drop unnamed columns and rows for non-participants, convert fastest laps to datetimes
        for name, table_df in self.tables.items():

            table_df = table_df.apply(lambda s: s.str.strip(' \'"'), axis=1)
            table_df = table_df.rename(columns=lambda c: c.strip(' \'"'))
            table_df = table_df.apply(lambda s: s.str.replace('(.*\d\d*\.\d{3})0$', r'\1'), axis=1)

            table_df["Pos"] = table_df["Pos"].astype(int)
            table_df = table_df.set_index("Pos")
            
            table_df["Laps"] = table_df["Laps"].astype(int)

            table_df = table_df[table_df["Laps"] > 0]

            table_df = table_df[table_df.columns.drop(list(table_df.filter(regex='Unnamed*')))]
            table_df = table_df.dropna(how='all', axis='columns')
            
            table_df["Best lap time"] = pd.to_datetime(table_df["Best lap"], format="%M:%S.%f", errors='coerce')

            self.tables[name] = table_df
            
            
        # Merge points column to both quali and race sessions, for race sessions: get attatched qualifying or previous race to get starting positions and show (probable) DNFs
        for name, table_df in self.tables.items():

            if name.startswith("Qualify"):

                points_column = pd.Series(np.zeros(len(table_df))).add(self.points_table["qualy_points"].astype(int), fill_value=0)
                table_df["Points"] = points_column.astype(int)

            if name.startswith("Race"):

                table_df["Time/Retired"] = table_df["Time/Retired"].str.replace('^\s*$', "")
                table_df["Consistency"] = table_df["Consistency"].str.replace('^-$', "")

                points_column = pd.Series(np.zeros(len(table_df))).add(self.points_table["points"].astype(int), fill_value=0)
                table_df["Points"] = points_column.astype(int)

                session_num = int(re.findall('(?:Qualify|Race)\s(\d+)\sresult|$', name)[0])
                qualy_table_name = "Qualify result" if session_num==1 and "Qualify result" in self.tables.keys() else "Qualify {} result".format(session_num)
                previous_race_table_name = "Race {} result".format(session_num - 1)
                if qualy_table_name in self.tables:
                    table_df = merge_qualy_info(table_df, self.tables[qualy_table_name], add_qualy_points=True)
                elif previous_race_table_name in self.tables:
                    table_df = merge_qualy_info(table_df, self.tables[previous_race_table_name])
                else:
                    table_df["Grid"] = ""
                    
                table_df["DNF"] = table_df["Laps"] < table_df["Laps"].max() - 3

            if self.debug: display(table_df)
            self.tables[name] = table_df
        

    # Generate quali table markdown from quali session dataframe, result is a list of lines
    def __generate_qualy_table_strings(self, table_name):

        table_df = self.tables[table_name]

        lines_buffer = [table_name, self.qualy_row_0, self.qualy_row_1]

        for i, df_row in table_df.iterrows():

            driver = df_row["Driver"]

            driver_info = self.drivers_table.loc[driver]

            position = i
            number = driver_info["number"]
            flag = driver_info["flag"]
            name = driver_info["name"]
            team = driver_info["team"]
            time = df_row["Best lap"]
            if position==1:
                time = "**{}**".format(time)

            line = self.qualy_row_2.format(position, number, flag, name, team, time) if i==0 else self.qualy_row.format(position, number, flag, name, team, time)
            lines_buffer.append(line)

        lines_buffer.append(self.qualy_row_last.format(self.simresults_url))

        if self.debug: print("made {} rows for table: {}".format(len(lines_buffer), table_name))
        return lines_buffer
    
    
    def __generate_race_table_strings(self, table_name):
    
        table_df = self.tables[table_name]

        lines_buffer = [table_name, self.race_row_0, self.race_row_1]

        for i, df_row in table_df.iterrows():

            driver = df_row["Driver"]

            driver_info = self.drivers_table.loc[driver]

            position = i
            number = driver_info["number"]
            flag = driver_info["flag"]
            name = driver_info["name"]
            team = driver_info["team"]
            laps = df_row["Laps"]
            points = df_row["Points"]
            timeorretired = df_row["Time/Retired"] if df_row["Time/Retired"] else "DNF"
            grid = df_row["Grid"] if df_row["Grid"]>0 else "DNQ"

            if "Qualify Points" in table_df.columns:
                points = str(df_row["Points"] + df_row["Qualify Points"]) + ("^^{}^^".format(grid) if df_row["Qualify Points"]>0 else "")

            line = self.race_row_2.format(position, number, flag, name, team, laps, timeorretired, grid, points) if i==0 else self.race_row.format(position, number, flag, name, team, laps, timeorretired, grid, points)
            lines_buffer.append(line)

        lines_buffer.append(self.race_row_last.format(self.simresults_url))
        
        fastest_driver = table_df.loc[table_df["Best lap time"]==table_df["Best lap time"].min()].iloc[0]["Driver"]
        fastest_driver_flag = self.drivers_table.loc[fastest_driver, "flag"]
        
        fastest_time = table_df.loc[table_df["Driver"]==fastest_driver]["Best lap"].item()
        lines_buffer.append(self.race_row_fastestlap.format(fastest_driver_flag, fastest_driver, fastest_time))

        if self.debug: print("made {} rows for table: {}".format(len(lines_buffer), table_name))
        return lines_buffer
    
    
    # Generate table markdown for all table_names, result is map from table names to table markdown string
    def generate_tables_strings(self):
        tables_strings = {}

        for name in self.table_names:
            lines_buffer = ["error!"]
            if name.startswith("Qualify"):
                lines_buffer = self.__generate_qualy_table_strings(name)
            elif name.startswith("Race"):
                lines_buffer = self.__generate_race_table_strings(name)
            tables_strings[name] = "\n".join(lines_buffer) + "\n\n"
            
        return tables_strings
    
    
    # Write table markdown for all table_names, optionally provide the generated table strings
    def write_generated_tables(self, tables_strings=None):
        if not tables_strings:
            tables_strings = self.generate_tables_strings()
            
        with open(self.output_file, "w+") as fp:
            for name in self.table_names:
                fp.write(tables_strings[name])
                if self.debug: print("wrote table {} to {}".format(name, self.output_file))

    

### Read Series Info

In [3]:
series = "PWC 2020"
series_sessions = ["Qualify result", "Race 1 result", "Race 2 result"]
series_race_sessions = [session for session in series_sessions if session.startswith("Race")]

series_drivers_table = RaceReport.read_drivers_table(series)
display(series_drivers_table)

series_points_table = RaceReport.read_points_table(series)
display(series_points_table)

series_tracks_table = RaceReport.read_tracks_table(series)
display(series_tracks_table)

,name,number,team,flag
ign,,,,
McLarenTim,McLarenTim,71,Timberwolves Racing,https://upload.wikimedia.org/wikipedia/en/thum...
PhotonBurst,PhotonBurst,28,Timberwolves Racing,https://upload.wikimedia.org/wikipedia/commons...
gunciz,Gunciz,000,Phoenix Racing Team Gold,https://upload.wikimedia.org/wikipedia/commons...
MrGrinder,MrGrinder,20,Phoenix Racing Team Gold,https://upload.wikimedia.org/wikipedia/en/thum...
Trempale,Trempale,17,Monterey SimSport,https://upload.wikimedia.org/wikipedia/en/thum...
JEMknight657,JEMknight657,15,Independent,https://upload.wikimedia.org/wikipedia/en/thum...
Waffles,Waffles,30,Independent,https://upload.wikimedia.org/wikipedia/en/thum...
Salzeder,Salzeder,16,DSW Racing,https://upload.wikimedia.org/wikipedia/en/thum...
NotJames,NotJames,73,Monterey SimSport,https://upload.wikimedia.org/wikipedia/en/thum...


,points,qualy_points
pos,,
1,40,3
2,36,2
3,32,1
4,28,0
5,25,0
6,22,0
7,19,0
8,16,0
9,13,0


,abbrev,full_name,flag,csv_manual_adjustment
directory,,,,
zolder,ZOL,Zolder,https://upload.wikimedia.org/wikipedia/commons...,-1
kyalami,KYA,Kyalami,https://upload.wikimedia.org/wikipedia/commons...,-1
cgv,VIL,Villeneuve,https://upload.wikimedia.org/wikipedia/commons...,-1
anderstorp,AND,Anderstorp,https://upload.wikimedia.org/wikipedia/en/thum...,-1
monza,MON,Monza,https://upload.wikimedia.org/wikipedia/en/thum...,-1
brno,BRN,Brno,https://upload.wikimedia.org/wikipedia/commons...,0
brands,BHA,Brands Hatch,https://upload.wikimedia.org/wikipedia/en/thum...,0
assen,ASS,Assen,https://upload.wikimedia.org/wikipedia/commons...,0
nurb,NUR,Nurburgring,https://upload.wikimedia.org/wikipedia/en/thum...,0


### Read Race Results

In [4]:
race_reports = {}

for race, race_row in series_tracks_table.iterrows():
    
    race_path = "{}/{}".format(series, race)
    if os.path.isdir(race_path):
        race_reports[race] = RaceReport(series_sessions, series, race, drivers_table=series_drivers_table, points_table=series_points_table, csv_manual_adjustment=race_row["csv_manual_adjustment"])
        print(race)
        [display(race_reports[race].tables[session]) for session in series_race_sessions]
    else:
        print(race, "not found")

zolder


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,Qualify Points,DNF
Pos,,,,,,,,,,,,,,,
1,,Timberwolves Racing,urd_t5_maures_2018,McLarenTim,16,22:45.433,01:24.273,99.32%,15,0,1900-01-01 00:01:24.273,40,1,3,False
2,,Timberwolves Racing,urd_t5_maures_2018,PhotonBurst,16,+00:18.490,01:24.917,98.69%,0,0,1900-01-01 00:01:24.917,36,3,1,False
3,,PRT Gold,urd_t5_aura_2018,MrGrinder,16,+00:23.170,01:24.941,98.49%,0,0,1900-01-01 00:01:24.941,32,5,0,False
4,,PRT Gold,urd_t5_aura_2018,gunciz,16,+00:31.415,01:25.042,97.86%,0,0,1900-01-01 00:01:25.042,28,4,0,False
5,,,urd_t5_aura_2018,Waffles,16,+00:44.662,01:26.230,98.62%,0,0,1900-01-01 00:01:26.230,25,6,0,False
6,,,urd_t5_bayro_2018,JEMknight657,16,+00:48.254,01:26.094,98.64%,0,0,1900-01-01 00:01:26.094,22,2,2,False
7,,M Power,urd_t5_bayro_2018,SBD,16,+00:51.262,01:26.023,97.90%,0,0,1900-01-01 00:01:26.023,19,7,0,False
8,,PRT Black,urd_t5_aura_2018,gman197002,16,+01:01.628,01:26.978,98.22%,0,0,1900-01-01 00:01:26.978,16,12,0,False
9,,,urd_t5_bayro_2018,NihonTiger,16,+01:15.478,01:27.660,98.31%,0,0,1900-01-01 00:01:27.660,13,11,0,False


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,DNF
Pos,,,,,,,,,,,,,,
1,,Timberwolves Racing,urd_t5_maures_2018,McLarenTim,29,41:37.353,01:24.279,99.15%,26,0,1900-01-01 00:01:24.279,40,1,False
2,,Timberwolves Racing,urd_t5_maures_2018,PhotonBurst,29,+00:16.476,01:24.339,98.60%,2,0,1900-01-01 00:01:24.339,36,2,False
3,,PRT Gold,urd_t5_aura_2018,MrGrinder,29,+00:21.628,01:24.684,98.84%,0,0,1900-01-01 00:01:24.684,32,3,False
4,,PRT Gold,urd_t5_aura_2018,gunciz,29,+00:25.517,01:24.834,98.93%,0,0,1900-01-01 00:01:24.834,28,4,False
5,,,urd_t5_bayro_2018,JEMknight657,29,+00:54.525,01:25.302,98.23%,0,0,1900-01-01 00:01:25.302,25,6,False
6,,M Power,urd_t5_bayro_2018,SBD,29,+00:57.661,01:25.581,98.73%,0,0,1900-01-01 00:01:25.581,22,7,False
7,,,urd_t5_aura_2018,Waffles,29,+01:20.885,01:25.954,97.87%,0,0,1900-01-01 00:01:25.954,19,5,False
8,,M Power,urd_t5_bayro_2018,bacsi97,29,+01:22.777,01:26.154,98.11%,0,0,1900-01-01 00:01:26.154,16,11,False
9,,PRT Black,urd_t5_aura_2018,gman197002,29,+01:23.901,01:26.068,97.97%,0,0,1900-01-01 00:01:26.068,13,8,False


kyalami


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,Qualify Points,DNF
Pos,,,,,,,,,,,,,,,
1,,,urd_t5_maures_2018,McLarenTim,14,22:07.556,01:33.545,99.67%,9,0,1900-01-01 00:01:33.545,40,2,2,False
2,,,urd_t5_maures_2018,PhotonBurst,14,+00:02.199,01:33.594,99.04%,4,0,1900-01-01 00:01:33.594,36,1,3,False
3,,,urd_t5_aura_2018,MrGrinder,14,+00:05.578,01:33.807,99.54%,0,0,1900-01-01 00:01:33.807,32,3,1,False
4,,,urd_t5_aura_2018,gunciz,14,+00:10.145,01:34.005,99.47%,0,0,1900-01-01 00:01:34.005,28,4,0,False
5,,,urd_t5_aura_2018,Waffles,14,+00:16.178,01:34.520,99.25%,0,0,1900-01-01 00:01:34.520,25,5,0,False
6,,,urd_t5_bayro_2018,bacsi97,14,+00:16.721,01:34.700,99.48%,0,0,1900-01-01 00:01:34.700,22,7,0,False
7,,,urd_t5_bayro_2018,SBD,14,+00:33.477,01:34.294,98.12%,0,0,1900-01-01 00:01:34.294,19,8,0,False
8,,,urd_t5_bayro_2018,NihonTiger,14,+00:39.760,01:36.190,99.28%,0,0,1900-01-01 00:01:36.190,16,9,0,False
9,,,urd_t5_bayro_2018,NotJames,14,+00:41.120,01:35.539,98.97%,0,0,1900-01-01 00:01:35.539,13,12,0,False


...

,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,Qualify Points,DNF
Pos,,,,,,,,,,,,,,,
1,,,urd_t5_maures_2018,McLarenTim,16,21:54.406,01:21.157,99.54%,13,0,1900-01-01 00:01:21.157,40,2,2,False
2,,,urd_t5_maures_2018,PhotonBurst,16,+00:00.807,01:20.979,99.16%,2,0,1900-01-01 00:01:20.979,36,1,3,False
3,,,urd_t5_aura_2018,MrGrinder,16,+00:03.672,01:21.057,99.14%,0,0,1900-01-01 00:01:21.057,32,4,0,False
4,,,urd_t5_aura_2018,gunciz,16,+00:09.666,01:21.570,99.35%,0,0,1900-01-01 00:01:21.570,28,3,1,False
5,,,urd_t5_aura_2018,Waffles,16,+00:11.325,01:21.240,98.81%,0,0,1900-01-01 00:01:21.240,25,6,0,False
6,,,urd_t5_bayro_2018,JEMknight657,16,+00:11.693,01:21.644,99.35%,0,0,1900-01-01 00:01:21.644,22,5,0,False
7,,,urd_t5_bayro_2018,NotJames,16,+00:32.565,01:22.597,98.89%,0,0,1900-01-01 00:01:22.597,19,9,0,False
8,,,urd_t5_aura_2018,xPsychedelix,16,+00:32.839,01:22.355,98.70%,0,0,1900-01-01 00:01:22.355,16,7,0,False
9,,,urd_t5_aura_2018,gman197002,16,+00:34.636,01:22.302,98.40%,0,0,1900-01-01 00:01:22.302,13,8,0,False


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,DNF
Pos,,,,,,,,,,,,,,
1,,,urd_t5_maures_2018,McLarenTim,30,41:24.378,01:21.288,99.47%,25,0,1900-01-01 00:01:21.288,40,1,False
2,,,urd_t5_maures_2018,PhotonBurst,30,+00:04.708,01:21.149,99.13%,4,0,1900-01-01 00:01:21.149,36,2,False
3,,,urd_t5_bayro_2018,JEMknight657,30,+00:11.107,01:21.436,99.29%,0,0,1900-01-01 00:01:21.436,32,6,False
4,,,urd_t5_aura_2018,MrGrinder,30,+00:11.636,01:21.375,99.15%,0,0,1900-01-01 00:01:21.375,28,3,False
5,,,urd_t5_aura_2018,gunciz,30,+00:12.662,01:21.224,99.05%,0,0,1900-01-01 00:01:21.224,25,4,False
6,,,urd_t5_aura_2018,Waffles,30,+00:26.831,01:21.708,98.91%,0,0,1900-01-01 00:01:21.708,22,5,False
7,,,urd_t5_aura_2018,xPsychedelix,30,+00:49.221,01:22.036,98.63%,0,0,1900-01-01 00:01:22.036,19,8,False
8,,,urd_t5_bayro_2018,NotJames,30,+00:53.287,01:22.024,98.52%,0,0,1900-01-01 00:01:22.024,16,7,False
9,,,urd_t5_aura_2018,gman197002,30,+00:54.089,01:22.180,98.37%,0,0,1900-01-01 00:01:22.180,13,9,False


hock


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,Qualify Points,DNF
Pos,,,,,,,,,,,,,,,
1,,,urd_t5_maures_2018,McLarenTim,15,22:28.346,01:29.169,99.77%,14,0,1900-01-01 00:01:29.169,40,1,3,False
2,,,urd_t5_maures_2018,PhotonBurst,15,+00:01.347,01:29.111,99.70%,0,0,1900-01-01 00:01:29.111,36,2,2,False
3,,,urd_t5_aura_2018,gunciz,15,+00:18.071,01:30.014,99.60%,0,0,1900-01-01 00:01:30.014,32,3,1,False
4,,,urd_t5_aura_2018,MrGrinder,15,+00:18.866,01:29.700,99.24%,0,0,1900-01-01 00:01:29.700,28,4,0,False
5,,,urd_t5_bayro_2018,JEMknight657,15,+00:19.482,01:29.663,99.35%,0,0,1900-01-01 00:01:29.663,25,6,0,False
6,,,urd_t5_aura_2018,Waffles,15,+00:28.873,01:30.027,98.95%,0,0,1900-01-01 00:01:30.027,22,7,0,False
7,,,urd_t5_aura_2018,gman197002,15,+00:37.582,01:30.864,99.40%,0,0,1900-01-01 00:01:30.864,19,11,0,False
8,,,urd_t5_bayro_2018,Salzeder,15,+00:38.589,01:30.552,99.01%,0,0,1900-01-01 00:01:30.552,16,8,0,False
9,,,urd_t5_bayro_2018,Trempale,15,+00:44.545,01:29.873,97.31%,0,0,1900-01-01 00:01:29.873,13,5,0,False


,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Best lap time,Points,Grid,DNF
Pos,,,,,,,,,,,,,,
1,,,urd_t5_maures_2018,McLarenTim,28,42:14.763,01:29.104,98.64%,23,0,1900-01-01 00:01:29.104,40,1,False
2,,,urd_t5_maures_2018,PhotonBurst,28,+00:28.398,01:29.157,98.72%,4,0,1900-01-01 00:01:29.157,36,2,False
3,,,urd_t5_aura_2018,MrGrinder,28,+00:31.887,01:29.675,98.04%,0,0,1900-01-01 00:01:29.675,32,4,False
4,,,urd_t5_bayro_2018,Trempale,28,+00:41.427,01:30.159,98.46%,0,0,1900-01-01 00:01:30.159,28,9,False
5,,,urd_t5_bayro_2018,JEMknight657,28,+00:41.871,01:29.822,98.20%,0,0,1900-01-01 00:01:29.822,25,5,False
6,,,urd_t5_aura_2018,Waffles,28,+00:50.833,01:30.462,98.27%,0,0,1900-01-01 00:01:30.462,22,6,False
7,,,urd_t5_aura_2018,gman197002,28,+01:09.598,01:30.906,98.19%,0,0,1900-01-01 00:01:30.906,19,7,False
8,,,urd_t5_bayro_2018,Salzeder,28,+01:10.441,01:30.775,97.99%,0,0,1900-01-01 00:01:30.775,16,8,False
9,,,urd_t5_bayro_2018,NotJames,28,+01:11.550,01:30.846,98.36%,0,0,1900-01-01 00:01:30.846,13,11,False


### Construct Points Table

In [5]:
points_table = pd.DataFrame(index=series_drivers_table.index, columns=pd.MultiIndex.from_product([series_tracks_table.index, series_race_sessions], names=["track", "session"]))

# loop through drivers index
# construct series with the multindex
# go thorugh race reports and count up points

for driver in points_table.index:
    for track, session in points_table.columns:
        
        # needs an class or smth smh
        # (pos, points, qualify points, DNF)
        result_tuple = (-1, 0, -1, 0, False)

        if track in race_reports:
            
            race_table = race_reports[track].tables[session]
            driver_race_result = race_table[race_table["Driver"]==driver]

            if len(driver_race_result==1):
                pos = driver_race_result.index[0]
                points = driver_race_result["Points"].item()
                qualify_pos = driver_race_result["Grid"].item()
                qualify_points = driver_race_result["Qualify Points"].item() if "Qualify Points" in driver_race_result.columns else 0
                dnf = driver_race_result["DNF"].item()
                
                result_tuple = (pos, points, qualify_pos, qualify_points, dnf)
        
        points_table.loc[driver, (track, session)] = result_tuple
        
# keep seperate table with points totals

tracks = points_table.columns.unique(level="track")
points_totals_table = pd.DataFrame(index=series_drivers_table.index, columns=tracks)

def get_total_weekend_points(driver_session_results):
    # sum race session points and qualify points
    return driver_session_results.apply(lambda t: t[1] + t[3]).sum()

for track in tracks:
    weekend_points = points_table[track]
    points_totals_table[track] = weekend_points.apply(get_total_weekend_points, axis=1)

points_totals_table["total"] = points_totals_table.agg(sum, axis=1)
points_totals_table["total_with_drop_week"] = points_totals_table.apply(lambda driver_row: driver_row["total"] - (min(driver_row)), axis=1)

points_totals_table = points_totals_table.sort_values("total_with_drop_week", ascending=False)
points_table = points_table.reindex(points_totals_table.index)

display(points_totals_table)
display(points_table)

track,zolder,kyalami,cgv,anderstorp,monza,brno,brands,assen,nurb,hock,total,total_with_drop_week
ign,,,,,,,,,,,,
McLarenTim,83,82,67,83,75,83,83,83,82,83,804,737
PhotonBurst,73,71,82,65,78,74,70,74,75,74,736,671
gunciz,56,56,69,74,69,56,54,65,54,41,594,553
MrGrinder,64,69,60,0,16,65,52,8,60,60,454,454
Waffles,44,47,41,56,44,0,50,56,47,44,429,429
gman197002,29,0,27,47,44,50,57,50,26,38,368,368
NotJames,2,26,0,0,53,44,38,35,35,21,254,254
NihonTiger,23,26,38,38,47,0,38,20,12,0,242,242
Trempale,12,33,47,47,0,0,0,38,20,41,238,238


track                           zolder                         \
session                  Race 1 result          Race 2 result   
ign                                                             
McLarenTim        (1, 40, 1, 3, False)   (1, 40, 1, 0, False)   
PhotonBurst       (2, 36, 3, 1, False)   (2, 36, 2, 0, False)   
gunciz            (4, 28, 4, 0, False)   (4, 28, 4, 0, False)   
MrGrinder         (3, 32, 5, 0, False)   (3, 32, 3, 0, False)   
Waffles           (5, 25, 6, 0, False)   (7, 19, 5, 0, False)   
gman197002       (8, 16, 12, 0, False)   (9, 13, 8, 0, False)   
NotJames           (14, 2, 9, 0, True)  (-1, 0, -1, 0, False)   
NihonTiger       (9, 13, 11, 0, False)  (10, 10, 9, 0, False)   
Trempale         (13, 4, 10, 0, False)  (11, 8, 13, 0, False)   
JEMknight657      (6, 22, 2, 2, False)   (5, 25, 6, 0, False)   
marzen224        (12, 6, 14, 0, False)  (13, 4, 12, 0, False)   
xPsychedelix     (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
SBD               (7, 19, 7, 0, False)   (6, 22, 7, 0, False)   
Salzeder         (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
bacsi97           (11, 8, 8, 0, False)  (8, 16, 11, 0, False)   
Viperion_NZ     (10, 10, 13, 0, False)  (12, 6, 10, 0, False)   
monochromatic    (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
Horanjo          (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
Phin             (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
Monochromatica   (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
swordhaven       (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   

track                         kyalami                         \
session                 Race 1 result          Race 2 result   
ign                                                            
McLarenTim       (1, 40, 2, 2, False)   (1, 40, 1, 0, False)   
PhotonBurst      (2, 36, 1, 3, False)   (3, 32, 2, 0, False)   
gunciz           (4, 28, 4, 0, False)   (4, 28, 4, 0, False)   
MrGrinder        (3, 32, 3, 1, False)   (2, 36, 3, 0, False)   
Waffles          (5, 25, 5, 0, False)   (6, 22, 5, 0, False)   
gman197002      (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
NotJames        (9, 13, 12, 0, False)   (9, 13, 9, 0, False)   
NihonTiger       (8, 16, 9, 0, False)  (10, 10, 8, 0, False)   
Trempale        (11, 8, 10, 0, False)  (5, 25, 11, 0, False)   
JEMknight657    (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
marzen224       (12, 6, 11, 0, False)   (11, 8, 12, 0, True)   
xPsychedelix    (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
SBD              (7, 19, 8, 0, False)   (7, 19, 7, 0, False)   
Salzeder        (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
bacsi97          (6, 22, 7, 0, False)   (8, 16, 6, 0, False)   
Viperion_NZ     (10, 10, 6, 0, False)  (-1, 0, -1, 0, False)   
monochromatic   (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
Horanjo         (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
Phin            (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
Monochromatica  (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
swordhaven      (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   

track                             cgv                         \
session                 Race 1 result          Race 2 result   
ign                                                            
McLarenTim       (4, 28, 1, 3, False)   (2, 36, 4, 0, False)   
PhotonBurst      (1, 40, 2, 2, False)   (1, 40, 1, 0, False)   
gunciz           (2, 36, 3, 1, False)   (3, 32, 2, 0, False)   
MrGrinder        (3, 32, 5, 0, False)   (4, 28, 3, 0, False)   
Waffles          (8, 16, 6, 0, False)   (5, 25, 8, 0, False)   
gman197002       (7, 19, 8, 0, False)    (11, 8, 7, 0, True)   
NotJames        (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
NihonTiger       (6, 22, 7, 0, False)   (8, 16, 6, 0, False)   
Trempale         (5, 25, 4, 0, False)   (6, 22, 5, 0, False)   
JEMknight657    (-1, 0, -1, 0, False)  (-1, 0, -1, 0, False)   
marzen224       (10, 10, 9, 0, False)  (7, 19, 10, 0, False)   
xPsych

### Print Driver's Standings

##### Header

In [6]:
pos_width = 55
driver_width = 170
result_width = 45
track_width = len(series_race_sessions)*result_width
points_width = 92
table_width = pos_width + driver_width + len(series_tracks_table)*track_width + points_width

In [7]:
header_0_format = """(% border="1" cellpadding="1" style="width:{table_width}px"%)"""
header_1_pos_driver_format = """|=(% colspan="1" rowspan="2" scope="row" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {pos_width}px" %)Pos|=(% colspan="1" rowspan="2" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {driver_width}px;" %)Driver"""
header_1_track_format = """|=(% colspan="2" rowspan="1" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {track_width}px" %)((({header_1_track_flag_and_abbrev})))"""
header_1_track_flag_and_abbrev_format = """
[[image:{track_flag}||height="14" width="23"]]

{track_abbrev}
"""
header_1_points_format = """|=(% colspan="1" rowspan="2" style="border-color: rgb(0, 0, 0); text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width: {points_width}px" %)Points"""
header_2_session_format = """|(% style="background-color:#eaecf0; text-align:center; vertical-align:middle; width:{result_width}px" %)**{session_abbrev}**"""

In [8]:
result_color_top_3 = ["#ffffbf", "#dfdfdf", "#ffdf9f"] 
result_color_points = "#dfffdf"
result_color_ret = "#efcfff"
result_color_default = "#ffffff"

In [9]:
driver_row_pos_format = """|=(% style="text-align: center; vertical-align: middle; background-color: rgb(234, 236, 240); width:{pos_width}px" %){pos}"""
driver_row_driver_format = """|(% style="width:{driver_width}px" %)[[image:{driver_flag}||height="14" width="23"]] {driver}"""
driver_row_result_format = """|(% style="background-color:{result_color}; text-align:center; vertical-align:middle; width:{result_width}px" %){result}"""
driver_row_points_format = """|(% style="text-align:center; vertical-align:middle; width:{points_width}px" %){points}"""

In [10]:
header_0 = header_0_format.format(table_width=table_width)

header_1_substrings = [header_1_pos_driver_format.format(pos_width=pos_width, driver_width=driver_width)]
for track in series_tracks_table.index:
    header_1_track_flag_and_abbrev = header_1_track_flag_and_abbrev_format.format(track_flag=series_tracks_table.loc[track]["flag"], track_abbrev=series_tracks_table.loc[track]["abbrev"])
    header_1_track = header_1_track_format.format(track_width=track_width, header_1_track_flag_and_abbrev=header_1_track_flag_and_abbrev)
    header_1_substrings.append(header_1_track)
header_1_substrings.append(header_1_points_format.format(points_width=points_width))
header_1 = "".join(header_1_substrings)

header_2_substrings = []
for _ in series_tracks_table.index:
    for session in series_race_sessions:
        session_number = re.findall('\d', session)[0]
        session_abbrev = "R{}".format(session_number)
        header_2_session = header_2_session_format.format(result_width=result_width, session_abbrev=session_abbrev)
        header_2_substrings.append(header_2_session)
header_2 = "".join(header_2_substrings)

lines_buffer = [header_0, header_1, header_2]

In [11]:
for pos, driver in enumerate(points_table.index):
    pos += 1
    
    driver_row_pos = driver_row_pos_format.format(pos_width=pos_width, pos=pos)
    
    driver_info = series_drivers_table.loc[driver]
    driver_flag = driver_info["flag"]
    driver_full_name = driver_info["name"]
    driver_row_driver = driver_row_driver_format.format(driver_width=driver_width, driver_flag=driver_flag, driver=driver_full_name)
    
    driver_row_substrings = [driver_row_pos, driver_row_driver]
    
    for result_tup in points_table.loc[driver]:
        result_pos, result_points, result_qualy_pos, result_qualy_points, result_dnf = result_tup
        
        result_string = ""
        result_color = result_color_default
        if result_pos > 0:
            result_string = str(result_pos)
            if result_points > 0:
                result_color = result_color_points
            if result_pos <= 3:
                result_color = result_color_top_3[result_pos-1]
            if result_dnf:
                result_string = "RET"
                result_color = result_color_ret
            if result_qualy_points > 0:
                result_string = "{}^^{}^^".format(result_string, result_qualy_pos)
        
        driver_row_result = driver_row_result_format.format(result_color=result_color, result_width=result_width, result=result_string)
        driver_row_substrings.append(driver_row_result)
        
    driver_totals = points_totals_table.loc[driver]
    driver_total_with_drop_week = driver_totals["total_with_drop_week"]
    driver_total = driver_totals["total"]
    points_string = "**{}**".format(driver_total_with_drop_week)
    if not driver_total_with_drop_week == driver_total:
        points_string = "{}^^{}^^".format(points_string, driver_totals["total"])
    driver_row_points = driver_row_points_format.format(points_width=points_width, points=points_string)
    driver_row_substrings.append(driver_row_points)
    
    driver_row = "".join(driver_row_substrings)
    lines_buffer.append(driver_row)

In [12]:
# TODO make the result tuple an object (3 places: where its set, totals lambda, and result_tup)

In [13]:
output_file = "{}/drivers_standings.txt".format(series)

with open(output_file, "w+") as fp:
    table_string = "\n".join(lines_buffer) + "\n\n"
    fp.write(table_string)

### Debug

In [14]:
# test_series_directory = "PWC 2020"
# test_race_directory = "hock"
# test_table_names = ["Qualify result", "Race 1 result", "Race 2 result"]

# testReport = RaceReport(test_table_names, test_series_directory, test_race_directory, csv_manual_adjustment=0, debug=True)
# # testReport.write_generated_tables()

In [15]:
# table_width = pos_width + driver_width + 1*track_width + points_width

# print(header_0.format(table_width=table_width))
# print(header_1_pos_driver.format(pos_width=pos_width, driver_width=driver_width), end="")
# header_zolder_track_flag_and_abbrev = header_track_flag_and_abbrev.format(track_flag="https://upload.wikimedia.org/wikipedia/commons/thumb/9/92/Flag_of_Belgium_%28civil%29.svg/23px-Flag_of_Belgium_%28civil%29.svg.png", track_abbrev="ZOL")
# print(header_1_track.format(track_width=track_width, header_track_flag_and_abbrev=header_zolder_track_flag_and_abbrev), end="")
# print(header_1_points.format(points_width=points_width))

# for session in series_race_sessions:
#     session_number = re.findall('\d', session)[0]
#     session_abbrev = "R{}".format(session_number)
#     print(header_2_session.format(result_width=result_width, session_abbrev=session_abbrev), end="")
    
# print()